In [1]:
import os
import re
import sys
sys.path.insert(0, "../../")
import pandas as pd

In [2]:
tonga_lsts = os.listdir(sys.path[0] + "data/tourism/tonga/scraping/")
filepaths = list()
for path in tonga_lsts:
    folder_path = sys.path[0] + "data/tourism/tonga/scraping/"
    if "Dec" in path:
        filepaths.append(folder_path + path)
    elif "2021" and "Bulletin" in path:
        filepaths.append(folder_path + path)
    else:
        pass

In [3]:
from src.scraper.parse import *

t = TesseractExtractor(filepaths[1])
text = t.extract(
    "Monthly Arrival and Departure by Mode of Travel and Net Migration",
    "--psm 3")

In [8]:
text = re.split(r'Issued by:', text)[0]
tables = re.split(r'\n{2,}', text)
rows = tables[1].split("\n")[2:]

visitor_stats = []
last_year = None
for idx, val in enumerate(rows):
    ## Get Year Match
    year = re.search(r'^(\d{4})', val)
    if year:
        last_year = int(year.group())
    if not year:
        elements = val.split(" ")[:5]
        elements = [
            re.sub(r'[^\w\s]', '', ele).strip() if ele != "-" else "0"
            for ele in elements
        ]
        elements = [ele if len(ele) != 0 else "0" for ele in elements]
        visitor_stats.append(elements)
vs_df = pd.DataFrame(visitor_stats,
                     columns=["month", "air", "ship", "yacht", "total"])
for col in vs_df.columns[1:]:
    vs_df[col] = vs_df[col].astype(int)
    

first_year = int(int(last_year) - len(vs_df)/12 + 1)
vs_df["validation"] = vs_df[["air", "ship", "yacht"]].sum(axis=1)


vs_df["date"] = pd.date_range(start=f'{first_year}/01/01', end=f'{last_year}/12/01', freq="MS")
## Quality Check
vs_df["pass"] = vs_df["validation"] == vs_df["total"]
## 
vs_df["doc_id"] = re.search(r'(\d{4})', filepaths[1]).group() + "-12"

In [13]:
t2 = TabulaExtractor(filepaths[0])

t2.extract("Monthly Arrival and Departure", table_page=0)

temp_columns = raw.iloc[0].tolist()
raw.columns = temp_columns
raw = raw.iloc[1:].reset_index(drop=True)

latest_year_idx, year_idxs, month_idxs = separate_years_months(raw, "Period")
months_df = raw.iloc[month_idxs].reset_index(drop=True)
start_year = detect_year(months_df.iloc[0])
months_df = months_df.iloc[:, 0:4].dropna(how="all").reset_index(drop=True)

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=6794Kb max_used=6794Kb free=124277Kb
 bounds [0x000000029a1d0000, 0x000000029a880000, 0x00000002a21d0000]
 total_blobs=2313 nmethods=1828 adapters=400
 compilation: disabled (not enough contiguous free space left)


NameError: name 'raw' is not defined

In [78]:
def separate_data(df: pd.DataFrame,
                  var: str,
                  split_rule: str):

    splited_lst = var.split(split_rule)
    var_number = len(splited_lst)

    obj = dict()
    for i in range(var_number):
        obj[str(splited_lst[i])] = []

    for i in df[var]:
        elems = i.split(" ")
        length = len(elems)
        if length == var_number:
            idx, var = 0, list(obj.keys())
            while idx < length:
                key, val = var[idx], elems[idx]
                obj[key].append(val)
                idx += 1

        elif length < var_number:
            idx, var = 0, list(obj.keys())
            print(elems)
            while idx < length and len(elems) != 0:
                key, val = var[idx], elems[idx].split(" ")[0]
                obj[key].append(val)
                elems = i.replace(val, "").strip()
                idx += 1
            else:
                key, val = var[idx], 0
                obj[key].append(val)
                idx += 1

        else:
            idx, var = 0, list(obj.keys())
            while idx < var_number:
                key, val = var[idx], elems[idx]
                obj[key].append(val)
                idx += 1
            else:
                key, val = var[-1], elems[idx]
                prev_val = obj[key][-1]
                obj[key][-1] = prev_val + val

    for i in range(var_number):
        df[str(splited_lst[i])] = obj[list(obj.keys())[i]]

    return df
separate_data(months_df, "Air Ship", " ")

['4']
['8']
['-']
['-']


,Period,Air Ship,Yacht,Total,Air,Ship
0,Jan,"4,294 3,346",-,"7 ,640","4,294","3,346"
1,Feb,"2,336 1,520",-,"3 ,856","2,336","1,520"
2,Mar,"2,835 3,062",14,"5,911","2,835","3,062"
3,Apr,"3,099 2,344",34,"5,477","3,099","2,344"
4,May,"4,348 627",1 66,"5 ,141","4,348",627
5,Jun,"4,785 3,318",305,"8 ,408","4,785","3,318"
6,Jul,"7,011 1,861",174,"9,046","7,011","1,861"
7,Aug,"5,855 -",2 52,"6 ,107","5,855",-
8,Sep,"5,061 -",2 56,"5,317","5,061",-
9,Oct,"4,504 4,746",2 95,"9,545","4,504","4,746"


In [151]:
def split_and_store_data(parts, key, obj):
    for i, v in enumerate(parts):
        obj[key[i]].append(v)
    return obj


def split_and_store_partial_data(parts, key, obj):
    for i, v in enumerate(key):
        if i < len(parts):
            obj[v].append(parts[i])
        else:
            obj[v].append(0)
    return obj

In [149]:
def separate_data(df: pd.DataFrame, var: str, split_rule: str) -> dict:
    """
    Split a column in a DataFrame into multiple columns based on a split rule.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        var (str): The name of the column to split.
        split_rule (str): The rule to split the values in the column.

    Returns:
        pd.DataFrame: The DataFrame with the split data in separate columns.
    """
    # Split the var column name using the split_rule

    split_values = var.split(split_rule)
    num_splits = len(split_values)

    split_data = {split: [] for split in split_values}
    print(split_data)
    # Iterate over the values in the var column
    for value in df[var]:

        elements = value.split(" ")
        length = len(elements)
        if length == num_splits:
            split_data = split_and_store_data(elements, split_values,
                                              split_data)
#             for i, v in enumerate(elements):
#                 split_data[split_values[i]].append(v)
        elif length > num_splits:
            if elements[-1] == "-" and length == num_splits + 1:
                new_elements = value.split("-")
                if len(new_elements) == num_splits:
                    for i, v in enumerate(new_elements):
                        split_data[split_values[i]].append(v.strip().replace(
                            " ", ""))

        else:
            split_data = split_and_store_partial_data(elements, split_values,
                                              split_data)


#             for i in range(num_splits):
#                 if i < length:
#                     split_data[split_values[i]].append(elements[i])
#                 else:
#                     split_data[split_values[i]].append(0)

    return split_data

c = separate_data(months_df, "Air Ship", " ")

{'Air': [], 'Ship': []}
['4']
['8']
['-']
['-']


In [150]:
pd.DataFrame(c)

,Air,Ship
0,"4,294","3,346"
1,"2,336","1,520"
2,"2,835","3,062"
3,"3,099","2,344"
4,"4,348",627
5,"4,785","3,318"
6,"7,011","1,861"
7,"5,855",-
8,"5,061",-
9,"4,504","4,746"


In [28]:


months = pd.DataFrame()

for file in filepaths[:-1]:
    print(file)

    df = load_pdf(file, "Monthly Arrival and Departure", table_page=-5)
    latest_year, year_idx, month_idx = split_time(df, "Period")
    month = df.iloc[month_idx, 0:4]
    start_year = detect_year(df.iloc[month_idx].iloc[0])

    month = (month.dropna(how="all").reset_index()
             .drop("index", axis=1))

    print(f"The file starts from {start_year}.")

    month = separate_data(month, "Air Ship", " ").drop("Air Ship", axis=1)
    month = remove_separator(month)
    month = month.replace(r'^\s*$', 0, regex=True)

    if check_quality(month, ["Period", "Year"], "Total") == False:
        name = file.split("/")[-1].split(".")[0]
        print("  ", name, "could go wrong!")

    generate_time(month, start_year)
    months = pd.concat([months, month], axis=0)

months = (months[["Year", "Period", "Air", "Ship", "Yacht", "Total"]]
          .drop_duplicates()
          .sort_values(by="Year")
          .reset_index()
          .drop("index", axis=1))

# Clean the datetime format
time = list()
for idx in months.index:
    month, year = months["Period"][idx], str(months["Year"][idx])
    if type(month) == str:
        try:
            YM = year + month
            time.append(datetime.strptime(YM, "%Y%B"))
        except:
            time.append(datetime.strptime(YM, "%Y%b"))
    else:
        time.append(month)

months["time"] = time
months = months.sort_values(by="time")

months

# Check for duplicates (e.g. Dec vs December in different years)
colnames = months.columns[~months.columns.isin(["Period"])]
indexes = months[colnames].drop_duplicates().index
months = months.iloc[indexes].reset_index().drop("index", axis=1)

# # Save the file
# months.to_csv("data/tourism/tonga/tonga_monthly_visitor.csv",
#               encoding="utf-8")


/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/scraping/Statistical-Bulletin-on-International-Arrivals-and-Departures-2021.pdf


NameError: name 'split_time' is not defined

In [18]:


error_lsts = list()
for file in tonga_lsts:
    if ".pdf" in file:
        print(f"{file} has started")

        try:
            filepath = os.getcwd() + "/data/tourism/tonga/" + file

            df = load_pdf(filepath, "Total Visitors by Usual Residence and Purpose of Visit",
                          table_page=5, table_seq=-1)

            df = df.dropna(how="all", axis=1)

            colname_lst = df.iloc[0].to_list()
            colname_lst[0:2] = ["Country", "Total"]
            df.columns = colname_lst

            df = separate_data(df, "Conference Friends").drop("Conference Friends", axis=1)
            df = (df.iloc[1:-2, :]
                    .dropna(how="all", axis=1)
                    .reset_index().
                    drop("index", axis=1))

            saved_path = os.getcwd() + "/data/tourism/tonga/temp/" + file.split(".")[0] + ".csv"

            df.to_csv(saved_path, encoding="utf-8")

        except:
            print(f"{file} has an error.")
            error_lsts.append(file)

with open(os.getcwd() + "/data/tourism/tonga/temp/error.txt", "w") as file:
    file.write("\n".join(error_lsts))

05-May-Migration-Report-2015.pdf has started
05-May-Migration-Report-2015.pdf has an error.
09-September-Migration-Report-2014.pdf has started
09-September-Migration-Report-2014.pdf has an error.
13-Annual-Migration-Report-2013.pdf has started
13-Annual-Migration-Report-2013.pdf has an error.
Migration-Report-Jun-2017.pdf has started
Migration-Report-Jun-2017.pdf has an error.
11-Migration-Report-Nov-2016.pdf has started
11-Migration-Report-Nov-2016.pdf has an error.
09-September-Migration-Report-2015.pdf has started
09-September-Migration-Report-2015.pdf has an error.
05-May-Migration-Report-2014.pdf has started
05-May-Migration-Report-2014.pdf has an error.
07-July-Migration-Report-2014.pdf has started
07-July-Migration-Report-2014.pdf has an error.
10-Migration-Report-Oct-2016.pdf has started
10-Migration-Report-Oct-2016.pdf has an error.
Migration-March-Report-2021.pdf has started
Migration-March-Report-2021.pdf has an error.
01-January-Migration-Report-2014.pdf has started
01-Janu